In [65]:
import numpy as np
import numba
from numba.typed import List

In [66]:
from base.Sheriff.env import *

In [67]:
getActionSize()

83

In [68]:
@numba.njit
def agent_random(state,temp,per):
    action = np.random.choice(np.where(getValidActions(state)==1)[0])
    per[0] = np.maximum(per[0],state)
    per[1] = np.minimum(per[1],state)
    return action,temp,per

In [69]:
perx = [np.full(getStateSize(),-100,dtype=np.float64),np.full(getStateSize(),100,dtype=np.float64)] # [max,min]

In [70]:
numba_main_2(agent_random,10000,perx,0)[0]

2525

In [71]:
@numba.njit
def agent_value(state,temp,per): # per = [max,min,value]
    action = np.random.choice(np.where(getValidActions(state)==1)[0])
    if getReward(state)!=-1:
        for i in range(getStateSize()):
            state_new = state.copy()
            state_new[i] = per[0][i]
            if getReward(state_new)==1 and  getReward(state)==0:
                per[2][i]+=1
            elif getReward(state_new)==0 and getReward(state)==1:
                per[2][i]-=1
            elif getReward(state_new)==-1 and getReward(state)==0:
                per[2][i]+=0.1
            elif getReward(state_new)==-1 and getReward(state)==1:
                per[2][i]-=0.1
        
            state_new2 = state.copy()
            state_new2[i] = per[1][i]
            if getReward(state_new2)==1 and getReward(state)==0:
                per[2][i]-=1
            elif getReward(state_new2)==0 and getReward(state)==1:
                per[2][i]+=1
            elif getReward(state_new2)==-1 and getReward(state)==0:
                per[2][i]-=0.1
            elif getReward(state_new2)==-1 and getReward(state)==1:
                per[2][i]+=0.1

    return action,temp,per


In [72]:
perx2 = perx
perx2.append(np.zeros(getStateSize(),dtype=np.float64))

In [73]:
numba_main(agent_value,agent_value,agent_value,agent_value,10000,perx)[0]

[2474, 2504, 2460, 2562]

In [74]:
value = perx2[2] / np.max(perx2[2])

In [75]:
@numba.njit
def agent_bias(state,temp,per): #per = [bias1,bias2,bias3,bias4,bias5,value1,value2,value3,value4,value5,turn,[score]]4
    turn  = per[10].astype(np.int64)[0]%5
    list_action = np.where(getValidActions(state)==1)[0]
    action = list_action[np.argmax(per[turn][list_action])]
    if getReward(state)!=-1:
        per[10][0]+=1
        per[5:10][turn][0]+= np.sum(state * value)
        if getReward(state)==1:
            per[11][turn]+=1
    
    return action,temp,per
    

In [86]:
per_bias = []
for i in range(5):
    per_bias.append(np.random.randn(getActionSize()))
for i in range(5):
    per_bias.append(np.array([0.]))
per_bias.append(np.array([0.]))
per_bias.append(np.zeros(5,dtype=np.float64))
    

In [87]:
numba_main_2(agent_bias,5000,per_bias,0)[0]

1367

In [88]:
state_value = per_bias[5:10]
for i in range(5):
    print(f'PLayer {i} : Rank: {5-i}; Win: {per_bias[11][np.argsort(per_bias[11])][i]:.0f}; State value: {np.array(state_value)[np.argsort(per_bias[11])][i][0]:.2f} ')

PLayer 0 : Rank: 5; Win: 0; State value: -149924.10 
PLayer 1 : Rank: 4; Win: 1; State value: -54849.69 
PLayer 2 : Rank: 3; Win: 273; State value: -16869.77 
PLayer 3 : Rank: 2; Win: 360; State value: 23341.81 
PLayer 4 : Rank: 1; Win: 733; State value: 73362.58 
